In [2]:
from datasets import load_dataset

In [3]:
dataset=load_dataset('ibm-research/argument_quality_ranking_30k', 'argument_quality_ranking')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

dev.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/20974 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3208 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6315 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf'],
        num_rows: 20974
    })
    validation: Dataset({
        features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf'],
        num_rows: 3208
    })
    test: Dataset({
        features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf'],
        num_rows: 6315
    })
})

In [5]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [6]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def tok(batch):
  return tokenizer(batch['argument'],truncation=True,max_length=256)

In [8]:
tdata = dataset.map(
    tok,
    batched=True,
    remove_columns=[c for c in dataset['train'].column_names if c not in ['argument','WA']]
)

Map:   0%|          | 0/20974 [00:00<?, ? examples/s]

Map:   0%|          | 0/3208 [00:00<?, ? examples/s]

Map:   0%|          | 0/6315 [00:00<?, ? examples/s]

In [9]:
tdata=tdata.rename_column('WA','labels')

In [10]:
tdata.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

In [11]:
tdata['train'][10]

{'labels': tensor(0.9011),
 'input_ids': tensor([  101,  1037,  2775,  3685,  2074,  3328,  2046,  1037, 25536, 16804,
          1998,  2131, 14203,  5970,  1010, 18643,  9619,  1998,  2966,  7040,
          2024,  1999,  2173,  2000,  5676,  2023,  2003,  2069,  4146,  9689,
          1998,  2006, 11007,  3572,  1997, 13366,  2953, 16383,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [12]:
from transformers import DataCollatorWithPadding

In [13]:
col=DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


In [15]:
from transformers import TrainingArguments,Trainer
import evaluate

In [16]:
model=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1,problem_type='regression')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
mse=evaluate.load('mse')
mae=evaluate.load('mae')

In [18]:
import numpy as np

In [22]:
def metric(eval_preds):
  logits,labels=eval_preds
  lpreds=np.squeeze(logits)
  labels=np.squeeze(labels)
  return{
      'mse':mse.compute(predictions=lpreds,references=labels)['mse'],
      'mae':mae.compute(predictions=lpreds,references=labels)['mae']
  }

In [25]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="mse",
    greater_is_better=False
)

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tdata["train"],
    eval_dataset=tdata["validation"],
    tokenizer=tokenizer,
    compute_metrics=metric,
)

trainer.train()

/tmp/ipython-input-722749021.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Mse,Mae
1,0.007800,0.041990,0.041990,0.155683
2,0.015400,0.036781,0.036781,0.141985
3,0.009800,0.037816,0.037816,0.144135


TrainOutput(global_step=3933, training_loss=0.011153751010055795, metrics={'train_runtime': 617.9356, 'train_samples_per_second': 101.826, 'train_steps_per_second': 6.365, 'total_flos': 1300925515973964.0, 'train_loss': 0.011153751010055795, 'epoch': 3.0})

In [27]:
results = trainer.evaluate(tdata["test"])
print(results)

{'eval_loss': 0.04037049040198326, 'eval_mse': 0.0403704897365842, 'eval_mae': 0.14992963934548773, 'eval_runtime': 18.4403, 'eval_samples_per_second': 342.457, 'eval_steps_per_second': 21.421, 'epoch': 3.0}


In [63]:
import torch
def arg_quali(args,model,tokenizer):
  inputs = tokenizer(args, truncation=True, padding=True, return_tensors="pt")
  device = next(model.parameters()).device
  inputs = {k: v.to(device) for k, v in inputs.items()}
  model.eval()
  with torch.no_grad():
      outputs = model(**inputs)
      preds = round(outputs.logits.squeeze().cpu().numpy()*4+1)
  return(f'Argument Rating:{preds}')


In [64]:
arg_quali('Homework improves student learning outcomes',model,tokenizer)

'Argument Rating:5'

In [65]:
pip install huggingface_hub

In [69]:
from huggingface_hub import login
login()

In [70]:
trainer.save_model("./argument_quality_regressor")
tokenizer.save_pretrained("./argument_quality_regressor")

('./argument_quality_regressor/tokenizer_config.json',
 './argument_quality_regressor/special_tokens_map.json',
 './argument_quality_regressor/vocab.txt',
 './argument_quality_regressor/added_tokens.json',
 './argument_quality_regressor/tokenizer.json')

In [74]:
trainer.push_to_hub('ByteMeHarder-404/argument_quality_ranking')

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /content/results/model.safetensors    :   0%|          | 14.2kB /  438MB            

  ...vents.1758556908.d6372781bad2.187.0:  11%|#         |   598B / 5.50kB            

  ...vents.1758557188.d6372781bad2.187.1:  11%|#         |   638B / 5.87kB            

  ...vents.1758557419.d6372781bad2.187.2:  11%|#         |   871B / 8.01kB            

  ...vents.1758558062.d6372781bad2.187.3:  11%|#         |  49.0B /   453B            

  /content/results/training_args.bin    :  11%|#         |   628B / 5.78kB            

CommitInfo(commit_url='https://huggingface.co/ByteMeHarder-404/results/commit/9c061fb4a5e3898992ffbe5acdb9bb6bd482c083', commit_message='ByteMeHarder-404/argument_quality_ranking', commit_description='', oid='9c061fb4a5e3898992ffbe5acdb9bb6bd482c083', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ByteMeHarder-404/results', endpoint='https://huggingface.co', repo_type='model', repo_id='ByteMeHarder-404/results'), pr_revision=None, pr_num=None)